In [ ]:
import os
from dotenv import load_dotenv

from src.agents.community_summarizer import CommunitiesSummarizer
from src.config import LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.factory.embeddings import get_embeddings
from src.graph.knowledge_graph import KnowledgeGraph

env = load_dotenv('config.env')

In [ ]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='vector'
)

embedder_conf = EmbedderConf(
    type=os.getenv("EMBEDDINGS_TYPE"),
    model=os.getenv("EMBEDDINGS_MODEL_NAME"),
)

model_conf=LLMConf(
    type=os.getenv("RE_MODEL_TYPE"),
    model=os.getenv("RE_MODEL_NAME"), 
    temperature=os.getenv("RE_MODEL_TEMPERATURE"), 
    deployment=os.getenv("RE_MODEL_DEPLOYMENT"),
    api_key=os.getenv("RE_API_KEY"),
    endpoint=os.getenv("RE_MODEL_ENDPOINT"),
    api_version=os.getenv("RE_MODEL_API_VERSION") or None
)

embeddings = get_embeddings(conf=embedder_conf)

kg = KnowledgeGraph(conf=kg_config,embeddings_model=embeddings)

In [ ]:
kg.louvain_modularity

In [ ]:
kg.leiden_modularity

In [ ]:
kg.number_of_leiden_communities # we are not skipping null values here

In [ ]:
kg.number_of_louvain_communities # we are not skipping null values here

In [ ]:
communities = kg.get_communities('louvain')

In [ ]:
communities

In [ ]:
communities[2].chunks

In [ ]:
summarizer = CommunitiesSummarizer(llm_conf=model_conf, embeddings_conf=embedder_conf)


In [ ]:
summarizer.get_community_report(community=communities[0])

In [ ]:
reports = summarizer.get_reports(communities)

reports

In [ ]:
kg.store_community_reports(reports)

### Retrieval of Communities
Now we are able to retrieve from two different vector stores: one dedicated to Chunks (Document vectors) and one for Summaries of Communities.  

Notice that once retrieved, the summary of the community has some metadata attached, such as the `community_id` and the `community_type`, while Chunks metadata could be used to retrieve neighbouring Chunks (`NEXT` relationship) and mentioned Entities (`MENTIONS` relationship).

In [ ]:
query = "Why is Europe introducting regulations on Apple?"

kg.cr_store.similarity_search_with_relevance_scores(query, k=2)

In [ ]:
query = "Why is Europe introducting regulations on Apple?"

kg.vector_store.similarity_search_with_relevance_scores(query, k=5)